# CHANGE DETECTION

## Instalar
Instalar geemap y eeconvert en Google Colab.

In [ ]:
!pip install geemap
!pip install eeconvert

## Importar

- Google Earth Engine y utilidades

In [1]:
import ee, eeconvert, geemap.eefolium as geemap

- Manejo de (Geo) Data Frames

In [2]:
import geopandas as gpd, pandas as pd

- Manejo de Arrays

In [3]:
import numpy as np

## Iniciar GEE
- Initialization

In [4]:
ee.Initialize()

## Datos

Directorio donde se encuentra el shapefile.

In [5]:
import os,glob
g = os.getcwd()
g
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

Lectura del shapefile y cambio del sistema de referencia.

In [6]:
points = gpd.read_file("Data_IDB/new/Zona_influencia_DR.shp").to_crs("EPSG:4326")

Asignación de un identificador único para cada registro.

In [7]:
points['ID'] = points['cuest_id_b'].astype(str) + points['parcela'].astype(str)

Obtener sólo los registros corregidos.

In [8]:
points = points[points['corregido2'].str.contains("ok")][['ID','patca','geometry']]

Convertir a Feature Collection.

In [9]:
eepoints = eeconvert.gdfToFc(points)

## Preprocesamiento de Imágenes

Nombres en para bandas en común entre Landsat 7 y Landsat 8. Esto con el fin de crear una única colección de imágenes con ambos datasets.

In [56]:
bandNames = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','pixel_qa']

Bandas correspondientes para cada satélite.

In [57]:
L7Bands = ['B1','B2','B3','B4','B5','B7','pixel_qa']
L8Bands = ['B2','B3','B4','B5','B6','B7','pixel_qa']

Lectura de cada uno de los datasets. Se realizan los filtros correspondientes por ubicación, fecha y se seleccionan únicamente las bandas en común (también se renombran).

In [58]:
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterBounds(eepoints).filterDate('2011-01-01','2016-01-01').select(L7Bands,bandNames)
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(eepoints).filterDate('2011-01-01','2016-01-01').select(L8Bands,bandNames)

Unir ambas colecciones en una sola y ordenar por fecha.

- creacion funciones de indices

In [59]:
def clouds_shadows_mask_L7(image):
    qa = image.select('pixel_qa')
#   If the cloud bit (5) is set and the cloud confidence (7) is high
#   or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = (qa.bitwiseAnd(1 << 5)
             .And(qa.bitwiseAnd(1 << 7))
             .Or(qa.bitwiseAnd(1 << 3)))
#  emove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    mask = image.updateMask(cloud.Not()).updateMask(mask2)
    return mask


def clouds_shadows_mask_L8(image):
#   // Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
#   // Get the pixel QA band.
    qa = image.select('pixel_qa');
#   // Both flags should be set to zero, indicating clear conditions.
    mask2 = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
             .And(qa.bitwiseAnd(cloudsBitMask).eq(0)))
    mask = image.updateMask(mask2) 
    return mask

def reflectance(image):
    return ee.Image(image.multiply(0.0001).copyProperties(image,["system:time_start"]))

def VI_L7(image):
    IVs = (image.addBands(image.normalizedDifference(['NIR', 'RED']).rename('NDVI')))
    IVs = (IVs.addBands(image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('NIR'),
      'RED': image.select('RED'),
      'BLUE': image.select('BLUE')}).rename('EVI')))
    IVs = (IVs.addBands(image.expression(
    '(NIR-RED)/(NIR+RED + 0.16)', {
      'NIR': image.select('NIR'),
      'RED': image.select('RED'),}).rename('SAVI')))
    return IVs

def preprocessing(imageCollection,Cloud_masking = True, calculateReflectance = True, CalculateVIs = True, Landsat = 8):
    if Cloud_masking:
        if Landsat == 8:
            imageCollection = imageCollection.map(clouds_shadows_mask_L8)
        if Landsat == 7:
            imageCollection = imageCollection.map(clouds_shadows_mask_L7)
    if calculateReflectance:
        imageCollection = imageCollection.map(reflectance)
    if CalculateVIs:
        imageCollection = imageCollection.map(VI_L7)
    return imageCollection

In [60]:
pos_L7 = preprocessing(L7,Landsat = 7)
pos_L8 = preprocessing(L8,Landsat = 8)
bandNames = ['NDVI','EVI','SAVI','BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

In [61]:
pos_L78 = pos_L7.merge(pos_L8).sort('system:time_start')

In [62]:
pos_L78_IV = pos_L78.select(bandNames)

## Detección de Cambios

### Algoritmo Continuous Change Detection and Classification
Este algoritmo recibe una colección de imágenes, utiliza todas las bandas para ajustar funciones armónicas y encontrar fechas de quiebre cuando las funciones cambian (el algoritmo funciona a base de pixel).

> **PUEDE HABER MÁS DE UNA FECHA DE QUIEBRE**

- Se utilizaron todas las bandas para el algoritmo (el algoritmo da la opción de usar sólo las bandas seleccionadas, así que este parámetro puede cambiarse a gusto).
- Se utilizaron las bandas VERDE y SWIR2 para enmascarar nubes y sombras usando el algoritmo TMASK-
- El formato de fecha de salida es MILISEGUNDOS.



In [63]:
CCDC = ee.Algorithms.TemporalSegmentation.Ccdc(collection = pos_L78_IV,dateFormat = 2) #tmaskBands = ['GREEN','SWIR2']

El algoritmo encuentra segmentos y guarda las fechas de acuerdo a segmentos:

- tStart: Inicio de segmento.
- tEnd: Fin de segmento.
- tBreak: Punto de quiebre (inicio del siguiente segmento: siguiente _tStart_. El fin de un segmento _tEnd_ no es necesariamente el inicio de otro segmento _tStart_ y por lo tanto no necesariamente un punto de quiebre _tBreak_).
- numObs: Observaciones del segmento.

Estos valores se extraen por cada registro del Feature Collection. Al ser polígonos, sólo nos interesa un valor (ee.Reducer.first()).

In [64]:
eepointsCCDC = CCDC.select(['tStart','tEnd','tBreak','numObs']).reduceRegions(collection = eepoints,reducer = ee.Reducer.first(),scale = 30,tileScale = 16)

Convertir valores extraídos a un Data Frame.

In [65]:
pointsCCDC = eeconvert.fcToGdf(eepointsCCDC)

C:\Users\fmontenegro\Miniconda3\envs\GEE\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Los valores se guardan en listas (el tamaño de la lista es proporcional al número de segmentos encontrados). Más de un segmento significa que un cambio fue encontrado. Estas listas son explotadas y apiladas en la misma columna.

In [66]:
tStart = pointsCCDC['tStart'].explode()
tEnd = pointsCCDC['tEnd'].explode()
tBreak = pointsCCDC['tBreak'].explode()
numObs = pointsCCDC['numObs'].explode()

Ya que las listas fueron explotadas y apiladas en la misma columna, los índices de los registros que se apilaron se utilizan para replicar los registros del data frame. Si una lista tiene dos segmentos para un registro, este debe ahora aparecer dos veces.

In [67]:
pointsExploded = pointsCCDC.loc[numObs.index,['ID','patca','geometry']]

Se añaden los valores de las listas explotadas al data frame donde se replicaron los registros. En el caso de los datos de fecha, estos son formateados de acuerdo a sus unidades de MILISEGUNDOS.

In [68]:
pointsExploded['tStart'] = pd.to_datetime(tStart,unit = 'ms')
pointsExploded['tEnd'] = pd.to_datetime(tEnd,unit = 'ms')
pointsExploded['tBreak'] = pd.to_datetime(tBreak,unit = 'ms')

En el caso del número de observaciones no se realiza ningún cambio.

In [69]:
pointsExploded['numObs'] = numObs

Cuando sólo existe un segmento, no existe (generalmente) punto de quiebre y su valor de MILISEGUNDOS es configurado en cero, lo que equivale a una fecha de 1970 (esto también ocurre a veces en el segmento final cuando hay más de un segmento). Así que podemos cambiar estos valores para evitar confusiones.

In [70]:
pointsExploded.loc[pointsExploded['tBreak'] < '2011-01-01','tBreak'] = np.nan

## Ejemplos

El registro con ID 386311 sólo tiene un segmento de 131 imágenes que inicia en 2011-01-01 y termina en 20019-12-17. No se ecnontró ningún punto de quiebre o cambio de cobertura en las imágenes.

In [71]:
pointsExploded[pointsExploded['ID'] == '386511']

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
29,386511,1,"POLYGON ((-71.68074 18.87789, -71.68202 18.877...",2011-01-01 15:01:56.470,2015-12-22 15:08:54.300,NaT,70


El registro con ID 396011 tiene 4 segmentos con 47, 42, 31 y 6 imágenes respectivamente. El primer segmento inicia en 2011-01-01 y termina en 2015-03-25. El segundo segmento inicia en 2015-04-10 y termina en 2018-01-28. Y así sucesivamente.

In [72]:
pointsExploded[pointsExploded['ID'] == '396011']

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2011-01-01 15:01:56.470,2015-04-26 15:08:03.960,2015-06-13 15:08:02.230,51
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2015-06-13 15:08:02.230,2015-12-22 15:08:54.300,NaT,12


In [73]:
# pointsExploded.to_csv("Data_IDB/CCDC_IV_2014.csv")
pointsExploded.head()

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
0,386311,1,"POLYGON ((-71.70753 18.89259, -71.70756 18.892...",2011-01-01 15:01:56.470,2015-12-22 15:08:54.300,NaT,76
1,386512,1,"POLYGON ((-71.68327 18.70962, -71.68331 18.709...",2011-01-01 15:01:56.470,2015-12-22 15:08:54.300,NaT,72
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2011-01-01 15:01:56.470,2015-04-26 15:08:03.960,2015-06-13 15:08:02.230,51
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2015-06-13 15:08:02.230,2015-12-22 15:08:54.300,NaT,12
3,407811,1,"POLYGON ((-70.88998 18.74631, -70.88973 18.746...",2011-01-01 15:01:56.470,2015-12-22 15:08:54.300,NaT,46
